# Authentifiction

In [1]:
from pwn import *
import json
import os

In [2]:
CHALLENGE_LENGTH    = 16
USER_NAME           = "AbyssOverlord"

HOST                = "challenges.challenge-ecw.eu"
PORT                = "38447"

In [3]:
def x(l1, l2):
    '''
    XORs 2 lists
    '''
    return [c1 ^ c2 for (c1, c2) in zip(l1, l2)]


def init_auth(s, challenge):
    request = {
        "option": "auth_init",
        "user_name": USER_NAME,
        "client_challenge": challenge.hex(),
    }
    s.sendline(json.dumps(request).encode())
    data = s.recv(1024, timeout=1)
    response = json.loads(data.decode())
    salt = response["salt"]
    server_challenge = bytes.fromhex(response["server_challenge"])
    server_proof = bytes.fromhex(response["server_proof"])
    return {"salt": salt, "challenge": server_challenge, "proof": server_proof}

In [4]:
conn = remote(HOST, PORT)
# OUR first challenge can be random, it doesn't matter
client_challenge = os.urandom(CHALLENGE_LENGTH)
r1 = init_auth(conn, client_challenge)
r1

[x] Opening connection to challenges.challenge-ecw.eu on port 38447
[x] Opening connection to challenges.challenge-ecw.eu on port 38447: Trying 141.94.137.179
[+] Opening connection to challenges.challenge-ecw.eu on port 38447: Done


{'salt': 'efca3f0a829c3eead647474b1c506226',
 'challenge': b'#%C]*c\x99\x96\x1b\x04\xbe\xb5\x02\xf3\x8ay',
 'proof': b'\x05 \xb1\xc9\x13]\xa7+f"\xb1\x1f\xfb\x81f\x19]\xd6,<\x85\x7fDl\x8a\xea\xd1;\xfb5\xab\xf4\xfe\x0f\x92\xc1h\xf9\xb7\x8b\xd7\x16D\x1c\x0eZ\xda\xf6fn\xc3'}

Here is the vulnerability in the challenge:

We now creates a new connection and kindly ask the server to encrypt the challenge for us.

Doing so provides us with a plaintext(`PT`)/ciphertext(`CT`) pair

In [5]:
server_challenge = r1["challenge"]
r2 = init_auth(remote(HOST, PORT), server_challenge)
r2

[x] Opening connection to challenges.challenge-ecw.eu on port 38447
[x] Opening connection to challenges.challenge-ecw.eu on port 38447: Trying 141.94.137.179
[+] Opening connection to challenges.challenge-ecw.eu on port 38447: Done


{'salt': 'efca3f0a829c3eead647474b1c506226',
 'challenge': b'?l7A\x82\xe9\x87$5t)\xf9q\t\xecS',
 'proof': b')\xde\xa6\xf4\xa4\xbc\xff\xd6\xda\xe2\x8f\x7f\xbd^\x83\xec\x98\xc1/M\xa1\x12V\xa8\x8d\x93\x01\x950[\xc87\xabS5\xc9R@\xf5\x0c\x1azK\xa7\x08-OJA\xae`'}

Because of how ECB works, we need to seperate the IV from the ciphered block, and xor the plaintext with the provided IV

In [6]:
proof = r2["proof"]
iv, CT = proof[:16], proof[16:32]
PT = x(list(server_challenge), iv)

print(bytes(PT))
print(bytes(CT))

b'\n\xfb\xe5\xa9\x8e\xdff@\xc1\xe61\xca\xbf\xad\t\x95'
b'\x98\xc1/M\xa1\x12V\xa8\x8d\x93\x01\x950[\xc87'


In [7]:
forged = proof[:-3] + bytes(CT) + bytes(x(PT, list(b"client")))

request = {"option": "auth_proof", "client_proof": forged.hex()}

conn.sendline(json.dumps(request).encode())
data = conn.recv(1024, timeout=1)
response = json.loads(data.decode())
response

{'authenticated': True, 'message': 'Welcome AbyssOverlord'}

In [8]:
request = {"option": "get_file"}
conn.sendline(json.dumps(request).encode())
file_content = b""
while True:
    data = conn.recv(1024, timeout=5)
    file_content += data
    if len(data) < 1024:
        break

conn.close()
print(file_content.decode())

[*] Closed connection to challenges.challenge-ecw.eu port 38447

Operation Blackout: A Strategic Assault on Europe's Internet Infrastructure

This document outlines the comprehensive plan devised to exploit vulnerabilities
in Europe's internet infrastructure, leveraging advanced AI tools to orchestrate
a large-scale cyber attack. The primary objective is to disrupt the continuity of
essential services, compromise the integrity of critical data lakes, destabilize
supply chains, and challenge the defense mechanisms of European nations.
This operation, codenamed Operation Blackout, is intended to instigate economic
and security chaos on a global scale.

Objectives
  - Disrupt Internet Services: Target key internet exchange points (IXPs)
    and submarine cables connecting Europe to the global internet.
  - Compromise Data Integrity: Infiltrate data lakes and corrupt or exfiltrate
    sensitive information.
  - Destabilize Supply Chains: Attack logistics and supply chain management systems

`ECW{authentifiction_9xniWVYniWzBmnq9cPG5FP1W}`